### Funções auxiliares

In [2]:
def formatar(x):
    """
    Formata um valor para impressão em uma matriz.

    Parâmetros:
    x: int or str - O valor a ser formatado.

    Retorna:
    str - Valor formatado, alinhado à direita e preenchido com espaços.
    """
    # Se o valor for um número inteiro, formata como inteiro alinhado à direita com um mínimo de 3 dígitos.
    if type(x) is int:
        return f"{x:>3d}"
    # Se o valor for uma string, formata como string alinhada à direita com um mínimo de 3 caracteres.
    else:
        return f"{x:>3}"

def print_matrix(S1, S2, M):
    """
    Imprime uma matriz formatada. Pode ser útil se quisermos analisar a matriz.

    Parâmetros:
    S1: list - Lista de caracteres para rótulos de coluna.
    S2: list - Lista de caracteres para rótulos de linha.
    M: list of list - Matriz a ser impressa.

    Retorna:
    None
    """
    # Obtém a largura dos rótulos de coluna
    col_width = max(len(str(x)) for x in S1)

    # Imprime os rótulos de coluna
    print(" " * (col_width + 2) + " ".join(formatar(x) for x in S1))

    # Imprime as linhas da matriz com rótulos de linha
    for x2, linha in zip(S2, M):
        # Imprime os rótulos de linha formatados à esquerda e preenchidos com espaços, seguidos pelos valores da linha formatados.
        print("{:<{}} {}".format(x2, col_width + 1, ' '.join(map(formatar, linha))))

    # Adiciona uma linha em branco após a impressão da matriz
    print()

    
def score_subst(x1, x2, g):
    """
    Calcula a pontuação de substituição entre dois caracteres.

    Parâmetros:
    x1: str - Primeiro caracter
    x2: str - Segundo caracter
    g: int - Penalidade por gap (espaço) na substituição

    Retorna:
    int - Pontuação da substituição entre os caracteres.
    """

    # Verifica se há um gap (espaço) em pelo menos um dos caracteres
    if '-' in x1 + x2:
        return g  # Retorna a penalidade por gap

    # Verifica se os caracteres fazem match, em caso afirmativo valor= 2 (exemplo)
    if x1 == x2:
        return 2  

    # Caso contrário, retorna a penalidade por mismatch valor=-1 (exemplo)
    return -1

# Algoritmo Smith Waterman

In [5]:
def smith_waterman(S1, S2, gap_penalty, score_subst):
    """
    Algoritmo de Smith-Waterman para calcular o alinhamento local de duas sequências.

    Parâmetros:
    S1: str - Primeira sequência
    S2: str - Segunda sequência
    gap_penalty: int - Penalidade por espaçamento (gap)
    score_subst: função auxiliar - Função que recebe dois caracteres e retorna o score de substituição entre eles

    Retorna:
    Tuple[int, Tuple[str, str]] - Pontuação do alinhamento ótimo e o par de sequências alinhadas
    """

    # Garantir que recebemos duas sequências string e que o gap penalty é inteiro.
    assert isinstance(S1, str) and isinstance(S2, str), "S1 e S2 devem ser strings."
    assert isinstance(gap_penalty, int), "A penalidade por gap deve ser um número inteiro."
    assert S1.strip() != '' and S2.strip() != '', "As sequências não podem ser vazias ou consistir apenas de espaços."
    assert all(c.isalpha() for c in S1), "A sequência S1 contém caracteres inválidos."
    assert all(c.isalpha() for c in S2), "A sequência S2 contém caracteres inválidos."

    # Adiciona um espaço '-' no início de ambas as sequências
    S2 = '-' + S2
    S1 = '-' + S1

    ncols = len(S1)
    nlins = len(S2)

    # Inicializa as matrizes de scores e de trace
    scores = [[0 for _ in range(ncols)] for _ in range(nlins)]
    trace = [['' for _ in range(ncols)] for _ in range(nlins)]

    # Preenche a matriz de scores e de trace tendo em conta que as primeiras linha e coluna já foram preenchidas
    for L in range(1, nlins):
        for C in range(1, ncols):
            # Calcula os valores para as três opções: diagonal, esquerda, cima
            diag = scores[L - 1][C - 1] + score_subst(S1[C], S2[L], gap_penalty)
            left = scores[L][C - 1] + gap_penalty
            up = scores[L - 1][C] + gap_penalty

            # Lista de escolhas e direções correspondentes
            choices = [diag, left, up]
            # Para conseguirmos colocar acessar às letras na matriz de trace ("DiagonalEsquerdaCima")
            directions = "DEC"

            # Encontra o máximo valor e a direção correspondente
            max_choice_index = choices.index(max(*choices))
            # Armazena na matriz de trace a direção escolhida para a célula
            trace[L][C] = directions[max_choice_index]
            # Armazena na matriz de pontuações a pontuação acumulada até uma célula com base na escolha da direção que maximizou a pontuação.
            scores[L][C] = max(0, choices[max_choice_index])

    # Se quisermos visualizar as matrizes (de score e de trace)
    print_matrix(S1, S2, scores)
    print_matrix(S1, S2, trace)
    
    # Encontrar a posição do máximo na matriz
    max_score = 0
    max_position = (0, 0)

    for i, row in enumerate(scores):
        for j, value in enumerate(row):
            if value >= max_score:
                max_score = value
                max_position = (i, j)


    # Reconstrução do alinhamento
    align1, align2 = "", ""
    L, C = max_position
    while L > 0 and C > 0 and scores[L][C] > 0:
        if trace[L][C] == 'D':
            align1 = S1[C] + align1
            align2 = S2[L] + align2
            L -= 1
            C -= 1
        elif trace[L][C] == 'E':
            align1 = S1[C] + align1
            align2 = '-' + align2
            C -= 1
        elif trace[L][C] == 'C':
            align1 = '-' + align1
            align2 = S2[L] + align2
            L -= 1

    # Retorna a pontuação total e o par de sequências alinhadas
    return max_score, (align1, align2)

In [6]:
#Exemplo de utilização

smith_waterman('AGACT', 'AGCAT', -1, score_subst)

     -   A   G   A   C   T
-    0   0   0   0   0   0
A    0   2   1   2   1   0
G    0   1   4   3   2   1
C    0   0   3   3   5   4
A    0   2   2   5   4   4
T    0   1   1   4   4   6

     -   A   G   A   C   T
-                         
A        D   E   D   E   E
G        C   D   E   E   E
C        C   C   D   D   E
A        D   C   D   E   D
T        C   D   C   D   D



(6, ('AG-ACT', 'AGCA-T'))

In [52]:
import unittest

class TestSmithWaterman(unittest.TestCase):


    def test_identical_sequences(self):
        self.assertEqual(smith_waterman('AGTACAT', 'AGTACAT', -1, score_subst), (14, ('AGTACAT', 'AGTACAT')))

    def test_completely_different_sequences(self):
        self.assertEqual(smith_waterman('AAAA', 'TTTT', -1, score_subst), (0, ('', '')))

    def test_sequences_with_small_difference(self):
        self.assertEqual(smith_waterman("CATCG", "CAGTC", -1, score_subst), (7, ('CA-TC', 'CAGTC')))

    def test_invalid_characters(self):
        with self.assertRaises(AssertionError):
            smith_waterman('AGTACAT', 'AGTAC&^%', -1, score_subst)
    
    def test_empty_sequences(self):
        with self.assertRaises(AssertionError):
            smith_waterman('', '', -1, score_subst)
            
    

# Executar os testes
result = unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestNeedlemanWunsch))

# Verificar que falhas houveram nos testes
if not result.wasSuccessful():
    for failure in result.failures:
        print(failure)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.012s

OK
